In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from xgboost import XGBClassifier
import joblib

In [2]:
df = pd.read_csv('/Users/wugaoyi/Learning/cs559/cs559_Final_Project/data/train_data.csv',na_values='?')
print(df.info())
# drop the non-relevant column
Index = df['Index']
df= df.drop(columns=['Index'])


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5807 entries, 0 to 5806
Data columns (total 97 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   Index                                                     5807 non-null   int64  
 1   Bankrupt?                                                 5807 non-null   int64  
 2    ROA(C) before interest and depreciation before interest  5807 non-null   float64
 3    ROA(A) before interest and % after tax                   5807 non-null   float64
 4    ROA(B) before interest and depreciation after tax        5807 non-null   float64
 5    Operating Gross Margin                                   5807 non-null   float64
 6    Realized Sales Gross Margin                              5807 non-null   float64
 7    Operating Profit Rate                                    5807 non-null   float64
 8    Pre-tax net Inter

In [3]:
rows,cols = df.shape
print(f'the number of rows before removal non-val data {df.isna().any(axis=1).sum()}')
df.dropna(inplace=True)
print(f'the number of rows before removal duplicated data {df.duplicated().sum()}')
df = df.drop_duplicates()
rows,cols = df.shape
# the data is very clean

the number of rows before removal non-val data 0
the number of rows before removal duplicated data 0


In [4]:
X = df.drop(columns=['Bankrupt?'])
y = df['Bankrupt?']

In [5]:
# standarization
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [6]:
# split the data into 4 sub-group
kmeans =KMeans(n_clusters=5,random_state=20)
kmeans.fit(X_scaled)
labels = kmeans.labels_
df_cluster = pd.DataFrame({
    'Index':df.index,
    'cluster':labels
})
print(df_cluster['cluster'].value_counts())
df = pd.concat([df,df_cluster],axis=1)
# df.to_csv('train_data_with_cluster',index=False)

for i in range(5):
    idx = (labels==i)
    subgroup = df[idx]
    print(f"the subgroup {i} has {subgroup['Bankrupt?'].sum()}")
    # subgroup.to_csv(f'subgroub{i}',index=False)


cluster
3    2163
1    1444
4    1350
0     479
2     371
Name: count, dtype: int64
the subgroup 0 has 105
the subgroup 1 has 4
the subgroup 2 has 0
the subgroup 3 has 59
the subgroup 4 has 30


In [7]:
pca = PCA(n_components=0.55)
X_pca = pca.fit_transform(X_scaled)
X_train, X_test, y_train, y_test = train_test_split(X_pca, labels, random_state=42)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, preds))
print(confusion_matrix(y_test,preds))


Accuracy: 0.9882920110192838
[[115   0   0   2   0]
 [  0 352   1   3   1]
 [  0   1  84   0   0]
 [  2   1   1 556   2]
 [  1   0   0   2 328]]


In [8]:
import joblib
import os

os.makedirs("generalization_model", exist_ok=True)

joblib.dump(pca, "generalization_model/training_pca.pkl")

joblib.dump(scaler, "generalization_model/training_scaler.pkl")

joblib.dump(clf, "generalization_model/training_classifier.pkl")

['generalization_model/training_classifier.pkl']

In [9]:
# df_test = pd.read_csv('./data/test_data.csv')
# Index = df_test['Index']
# df_test= df_test.drop(columns=['Index'])

In [10]:
# X_test = df_test
# X_test_np = scaler.transform(X_test)
# X_test_pca = pca.transform(X_test_np)
# labels=clf.predict(X_test_pca)
# X_test['cluster']=labels
# print(X_test['cluster'].value_counts())

# df_predict = pd.DataFrame({
#     'Index':Index,
#     'cluster':labels
# })
# df_predict['prediction'] = -1
# df_predict.loc[df_predict['cluster'] == 2, 'prediction'] = 0

# #extract the label 0
# idx_0 = df_predict['cluster'] == 0
# X_test_subgroup0 = X_test.loc[idx_0, :]
# subgroup0_extract_features = joblib.load("./subgroup0_model/extract_features.pkl")
# subgroup0_scaler = joblib.load("./subgroup0_model/scaler.pkl")
# subgroup0_model = joblib.load("./subgroup0_model/best_stack_model.pkl")
# X_test_subgroup0_selected =X_test_subgroup0[subgroup0_extract_features]
# X_test_subgroup0_scaled = subgroup0_scaler.transform(X_test_subgroup0_selected)
# preds_0 = subgroup0_model.predict(X_test_subgroup0_scaled)
# df_predict.loc[idx_0, 'prediction'] = preds_0
# df_predict['prediction'].value_counts()